# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv #no nned for ollama
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


# I will be using ollama

In [3]:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [15]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
MODEL = "llama3.2"

In [5]:
links = fetch_website_links("https://marufmullah50.github.io/")
links

['#about',
 '#education',
 '#research',
 '#projects',
 '#skills',
 '#experience',
 '#contact',
 '#cv',
 '#about',
 '#contact',
 'https://drive.google.com/file/d/1vpVZQZMdddl-lyYXsE03T3hKZWrSqqi2/view?usp=drive_link',
 'https://drive.google.com/file/d/18LPhozjqo-WKTq83NEwYE-vnDuJbnRpy/view?usp=sharing',
 'https://drive.google.com/file/d/1pXTrq3WtxePYmgT3U3wV0EFVgUJbA0YM/view?usp=drive_link',
 'https://github.com/marufmullah50/Casting-Defect-identify',
 'https://github.com/marufmullah50/Fish-detection-app',
 'https://github.com/marufmullah50/Photo-background-editor-Streamlit-App',
 'https://github.com/marufmullah50/Surface_Roughness_Prediction',
 'https://github.com/marufmullah50/Dhaka_Wind_Property_Prediction',
 'mailto:md.marufmullah50@gmail.com',
 'tel:+8801609099183',
 'https://www.linkedin.com/in/marufmullah50',
 'https://github.com/marufmullah50',
 'https://github.com/marufmullah50/CV_Resume']

## First step: Have ollama figure out which links are relevant

### Use a call to llama3.2 to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [20]:
print(get_links_user_prompt("https://marufmullah50.github.io/"))


Here is the list of links on the website https://marufmullah50.github.io/ -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

#about
#education
#research
#projects
#skills
#experience
#contact
#cv
#about
#contact
https://drive.google.com/file/d/1vpVZQZMdddl-lyYXsE03T3hKZWrSqqi2/view?usp=drive_link
https://drive.google.com/file/d/18LPhozjqo-WKTq83NEwYE-vnDuJbnRpy/view?usp=sharing
https://drive.google.com/file/d/1pXTrq3WtxePYmgT3U3wV0EFVgUJbA0YM/view?usp=drive_link
https://github.com/marufmullah50/Casting-Defect-identify
https://github.com/marufmullah50/Fish-detection-app
https://github.com/marufmullah50/Photo-background-editor-Streamlit-App
https://github.com/marufmullah50/Surface_Roughness_Prediction
https://github.com/marufmullah50/Dhaka_Wind_Property_Prediction
mailto:md.marufmullah50@gmail.com
tel:+880160

# for open ollama change openai with ollama 
```python
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=ollama,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
`````

In [8]:
def select_relevant_links(url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )

    result = response.choices[0].message.content
    links = json.loads(result)
    return links


In [23]:
select_relevant_links("https://marufmullah50.github.io/")

{'links': [{'type': 'about page', 'url': 'https://marufmullah50.github.io/'},
  {'type': 'contact page', 'url': 'https://marufmullah50.github.io/'},
  {'type': 'LinkedIn profile',
   'url': 'https://www.linkedin.com/in/marufmullah50'},
  {'type': 'GitHub profiles (Portfolio)',
   'url': 'https://github.com/marufmullah50'}]}

````python
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links
```

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [30]:
select_relevant_links("https://www.anthropic.com/")

Selecting relevant links for https://www.anthropic.com/ by calling llama3.2
Found 10 relevant links


{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'},
  {'type': 'company page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'economy page',
   'url': 'https://www.anthropic.com/economic-futures'},
  {'type': 'constitution and transparency page',
   'url': 'https://www.anthropic.com/constitution'},
  {'type': 'news and announcements page',
   'url': 'https://www.anthropic.com/news'},
  {'type': 'learn more page', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'trust center page', 'url': 'http://trust.anthropic.com/'}]}

In [29]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 11 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co'},
  {'url': '/brands'},
  {'type': 'About us page', 'url': './'},
  {'type': 'Company/Innovation pages', 'url': 'spaces'},
  {'type': 'Enterprise page', 'url': 'enterprise'},
  {'type': 'Pricing page', 'url': '/pricing'},
  {'type': 'Model cards ', 'url': 'https://huggingface.co/models'},
  {'type': 'Dataset page', 'url': 'https://huggingface.co/datasets'},
  {'url': '/models'},
  {'type': 'Login/Account pages', 'url': '/login'},
  {'type': 'Careers/Jobs page', 'url': '/join'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [10]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [32]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 11 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
moonshotai/Kimi-K2.5
Updated
about 8 hours ago
•
32.4k
•
1.24k
nvidia/personaplex-7b-v1
Updated
2 days ago
•
62.1k
•
1.52k
Tongyi-MAI/Z-Image
Updated
3 days ago
•
2.89k
•
717
deepseek-ai/DeepSeek-OCR-2
Updated
1 day ago
•
64.6k
•
575
Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
Updated
2 days ago
•
198k
•
806
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.04k
Qwen3-TTS Demo
🎙
1.04k
Transform text into natural-sounding speech with custom voices
Running
on
Zero
Featured
1.28k
Qwen Image Multiple Angles 3D Camera
🎥
1.28k
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.91k
Z Image Turbo
🖼
1.91k
Generate

In [11]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [35]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 6 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2.5\nUpdated\nabout 8 hours ago\n•\n32.4k\n•\n1.24k\nnvidia/personaplex-7b-v1\nUpdated\n2 days ago\n•\n62.1k\n•\n1.52k\nTongyi-MAI/Z-Image\nUpdated\n3 days ago\n•\n2.89k\n•\n717\ndeepseek-ai/DeepSeek-OCR-2\nUpdated\n1 day ago\n•\n64.6k\n•\n576\nQwen/Qwen3-TTS-12Hz-1.7B-CustomVoice\nUpdated\n2 days ago\n•\n198k\n•\n807\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\

In [13]:
def create_brochure(company_name, url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [40]:
create_brochure("Anthropic", "https://www.anthropic.com/")

Selecting relevant links for https://www.anthropic.com/ by calling llama3.2
Found 8 relevant links


**Anthropic: Empowering AI for Humanity's Long-Term Well-Being**

Welcome to Anthropic, a public benefit corporation dedicated to securing the benefits and mitigating the risks of artificial intelligence. Our mission is to build AI that serves humanity's long-term well-being.

**Our Values**

At Anthropic, we are guided by a set of core values:

* **AI Safety**: We believe that AI should be designed to prioritize human safety and well-being.
* **Responsible Scaling**: We strive to ensure that our technology is scalable in a responsible and transparent manner.
* **Humility**: We recognize the limits of our knowledge and are committed to continuous learning and improvement.

**Our Research**

Anthropic is at the forefront of AI research, with a focus on advancing the state-of-the-art in AI safety, explainability, and alignment. Our work includes:

* **Economic Futures**: Exploring the potential impacts of AI on the economy and society.
* **Commitments**: Defining our principles for responsible AI development and deployment.
* **Initiatives**: Engaging with stakeholders to develop and implement best practices for AI ethics.

**Our Products**

We are proud to offer a range of products and tools that advance the field of AI safety and alignment:

* **Claude**: Our flagship model, designed to provide interpretability and explainability of complex AI systems.
* **Claude Opus 4.5**: The most advanced model in the world for coding, agents, computer use, and enterprise workflows.

**Join Us**

At Anthropic, we are passionate about creating a better future through AI. Join us as we shape the future of artificial intelligence.

**Careers**

We offer exciting opportunities to work with our talented team:

* **Engineering**: Development of cutting-edge AI technologies.
* **Research**: Advancing the state-of-the-art in AI safety and alignment.
* **Academy**: Training and education for individuals interested in AI ethics and responsibility.

**Get In Touch**

Contact us to learn more about Anthropic's mission, research, and products.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

#here S this line sometimes tries to read .content when it doesn’t exist:

chunk.choices[0].delta.content
```python
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
```
# solution

In [14]:
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)


    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta and delta.content:
            response += delta.content
            update_display(
                Markdown(response),
                display_id=display_handle.display_id
            )
    return response


#Change in system prompt (2nd one)

In [15]:
Anthropic_english = stream_brochure("Anthropic", "https://www.anthropic.com/")

Selecting relevant links for https://www.anthropic.com/ by calling llama3.2
Found 15 relevant links


# Anthropic Brochure

Welcome to Anthropic, a pioneering public benefit corporation dedicated to harnessing the power of AI for humanity's long-term well-being.

## Our Mission

At Anthropic, we believe that Artificial Intelligence holds immense potential to shape the world. However, its benefits must be secured and risks mitigated. We're committed to building AI that prioritizes safety, interpretability, and alignment with human values.

### The Claude Advantage

Meet Claude, our cutting-edge AI platform that enables developers to build, deploy, and manage AI models at scale. With Claude Opus 4.5, the world's most advanced model for coding, agents, computer use, and enterprise workflows, we're redefining the future of AI development.

### What We're Working Towards

We're pushing the boundaries of what's possible with AI in several exciting areas:

* **Mars Exploration**: Our AI-powered systems have successfully assisted the first AI-assisted drive on another planet, opening up new frontiers for human exploration and discovery.
* **AI Safety**: Our research focuses on developling interpretable AI models that prioritize alignment with human values, ensuring AI serves humanity's long-term well-being.

### Our Culture

At Anthropic, we prioritize a culture of empathy, diversity, and inclusivity. We believe that by coming together, we can achieve more than any individual could alone. With our dedicated team of experts, you'll find:

* **Expertise**: From computer vision to natural language processing, our team has the expertise to tackle some of AI's most complex challenges.
* **Innovation**: Our commitment to innovation drives us to push the boundaries of what's possible with AI, resulting in new breakthroughs and discoveries.

### Join Our Journey

Ready to be part of a mission-driven company that's shaping the future of AI? Explore our careers page to learn about job opportunities and how you can contribute to our mission.

**Log in to Claude**
[Download app](https://example.com/download-app)
**Learn more about Claude**
[Website](https://anthropic.ai)

### Stay Connected

Sign up for our newsletter to stay updated on the latest developments, research updates, and news from Anthropic.

**Contact Us**
[Email](mailto:info@anthropic.ai)
[Social Media](https://facebook.com/anthropic.ai)

#Translating

In [16]:
from IPython.display import display, Markdown, update_display

def stream_translate_to_bengali(brochure_text):
    if not brochure_text:
        raise ValueError("Brochure text is empty. Cannot translate.")

    stream = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system",
             "content": "Translate the following text into natural Bengali. Preserve formatting and tone. Do not add explanations."},
            {"role": "user", "content": brochure_text}
        ],
        stream=True
    )

    translated_text = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta and delta.content:
            translated_text += delta.content
            update_display(Markdown(translated_text), display_id=display_handle.display_id)

    return translated_text

In [ ]:
paragraphs = Anthropic_english.split("\n\n")
bengali_brochure = ""
for p in paragraphs:
    bengali_brochure += stream_translate_to_bengali(p) + "\n\n"

আন্থ্রোপিক প্যাটার্নশীপ বর্ণালি 

মনে হয়, বিশ্বের সংগতিক খবর আমরা অন্যদের থেকে ভাল খাইব।

উইলকাম টু অ্যান্থ্রোপিক, একটি দর্শনীয় সামাজিক লাভ সংস্থা যার লক্ষ্য হ'ল আই নেটিভজের শক্তি ব্যবহার করে মানবজগতের দীর্ঘমেয়াদী উন্নতি।

 
আই সি

AI

## আমাদের উদ্যোগ
### সন্ধিপ্রিতিশ্চিত  ভবিষ্যত, একটি অনলাইন পরিষেবা উপজাত হিসেবে, আমাদের ঊর্ধ্বলোকে যাঁরা এটি খুবই গভীরভাবে অন্তর্দৃষ্টিপ্রাপ্ত হয়, আমরা তাদের উচিত এবং সক্ষম পছন্দ শোনাই এভাবে নিজেদেরকে সার্ভড করি যা অনন্য আর এফটি যোগ্য।

অ্যান্থ্রোপিকে বিশ্বাস করি যে অ্যার্টিফিশিয়াল ইন্টেলিজেন্স দুঃখিত বিশ্বকে পরিবূহ হতে পারে।  এর ফলে আসা সুবিধাগুলি নিশ্চিত করা এবং ঝুঁকি হ্রাস করা অপরিহার্য।  আমরা দৃঢ়ভাবে জড়িত, নির্ভিয়োগতা এবং মানুষের মূল্যবোধের সাথে সারিবরাও হওয়ার জন্য ভিজ্যুয়ালের এক-পাশ উপযোগী আই তৈরি করার অঙ্গীকার রাখি।

### ক্লোড সুবিধা 

### What is the Claude Advantage?
#### ক্লোদ অ্যাডভাংটেজ আপনি যে-কী?
### Our unique approach combines human judgment with AI-driven insights to deliver accurate and actionable information. 
 #### মানব জুগলতা এবং অ্যাই-ড্রাইভেড ইনসাইট, আপনি যে দরকার? 
### Unlike conventional solutions that rely solely on AI or human analysis, our approach combines the strengths of both to provide a comprehensive and accurate solution. 
#### ক্ষমতা এবং দুর্বলতার পিছনে যাওয়া সাধারণ অভিযোজনটি, আপনাকে যে মান দেয়?
### Learn how our unique approach combines the best of human judgment and AI-driven insights to deliver accurate and actionable information. 
#### আপনার ক্লোদ অ্যাডভাংটেজের অভিব্যখ্যান
### Stay ahead of the competition with our innovative solution that combines human judgment and AI-driven insights. 
#### প্রতিরোধ
ক্লোদ আপনাকে শক্তিশালীভাবে, নির্ভরযোগ্যভাবে, এবং একটি জ্ঞাত সিদ্ধান্ত গ্রহণের জন্য প্রস্তুত করা. 
আপনি যে চূড়ান্ত উপলব্ধি?

মিঠুনে আছে, পর্যায়ক্রমের এই বর্ধিত খবরা-খিত জ্ঞানি সিস্টেম যা উন্নয়নকারীদের স্কেলে ভড়ঘর প্রবর্তক, দক্ষিণাঙ্গ এন জিরো ফিট আই মডেল তৈরি এবং চলায বৃদ্ধি পেতে সক্ষম করে। ক্লুডিয়া ওপাস 4.5 শেখার দুনীয়তম মডেল, কোডিং, এজেন্ট, কম্পিউটার আচরণ এবং উৎপাদনিক স্বায়ত্তশাসনের লাগনায়,   আমরা AI উন্নয়নের ভবিষ্যৎকে পুনঃনির্মিত করে ।

### আমরা কীভাবে প্রতিশ্রুতি দিচ্ছি 

যোগাযোগ
দেওয়া: ৩-৬৫৩৮১–৭  [email protected]](mailto:support@yourwebsite.com )

উত্তরদাতার অনুরোধ: ডিরেক্টরিয়াল-মেইল (1)

রিভিউ
রিয়েস্ট আর্টিকল

সাহায্য
কন্টাকট আস
অ্যাপ্লিকেশন ফর্ম

আমাদের গ না কয়েমাত্র স্য

In [47]:
stream_brochure("Tritechbd", "https://www.tritechbd.com/")

Selecting relevant links for https://www.tritechbd.com/ by calling llama3.2
Found 8 relevant links


# Welcome to Tritechbd: Your Partner in HVAC Solutions

At Tritechbd, we're passionate about providing the best HVAC solutions for our customers across Bangladesh. Our journey began with a dream to deliver innovative and reliable cooling systems that meet the unique needs of different industries.

## Who We Are

As a leading HVAC supplier, we pride ourselves on being a trusted partner to various sectors, including:

* Hospitality
* Healthcare
* Education
* Government Buildings
* Food & Pharma

We're dedicated to helping businesses and organizations maintain optimal temperatures, ensuring comfort, productivity, and efficiency.

## Our Products and Solutions

Our comprehensive range of products includes:

* VRF AC Systems (Daikin VRV System, LG VRF System, Midea VRF System)
* Electric Chillers (Dunham Bush Chiller, Daikin Chillers, Midea Chillers)
* Industrial Dehumidifiers (Fisair Dehumidifier, YAKE Dehumidifier)
* Clean Room Solutions (AHU, Ventilation Fan, Refrigeration Systems)
* Modular Cold Rooms
* Air Conditioning Solutions for Hotels and Hospitals

## Our Culture

At Tritechbd, we value:

* **Innovation**: We're committed to staying at the forefront of HVAC technology.
* **Customer Satisfaction**: Your satisfaction is our top priority!
* **Teamwork**: Collaboration and communication are essential aspects of our team culture.
* **Quality**: We strive for excellence in every product and service.

## Join Our Team

Ready to be part of a dynamic team that makes a difference? Check out our current job openings: [link]

Get in touch with us today:

Email: support@tritechbd.com
Phone: +8801786337711
Address: JCX Business Tower, Plot - 1136/A, Japan Street, Block - I, Basundhara R/A, Dhaka 1229

Let's cool things down together!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>